In [ ]:
import margin_simulator as ms
import widgets_layout as wl 
from ipywidgets import interactive_output

In [ ]:
out = interactive_output(ms.run_simulation_and_analysis, {
    'initial_portfolio': wl.portfolio_slider,
    'initial_loan': wl.loan_slider,
    'annual_market_return': wl.market_slider,
    'annual_volatility': wl.volatility_slider,
    'annual_dividend_yield': wl.dividend_slider,
    'annual_interest_rate': wl.interest_slider,
    'years': wl.years_slider,
    'yearly_contribution': wl.contribution_slider,
    'yearly_withdrawal': wl.withdrawal_slider,
    'capitalize_interest': wl.capitalize_checkbox,
    'max_loan_to_value': wl.max_ltv_slider,
    'hf_token': wl.hf_token_input
})
display(wl.description)
display(wl.controls)
display(out)